In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation,ZeroPadding2D, MaxPool2D, Add, GlobalAveragePooling2D,Dense
from tensorflow.keras.models import Model 
from tensorflow.keras import Input

In [ ]:
(img_train, label_train), (img_test, label_test) = tfds.as_numpy(tfds.load(
    'Cifar10',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete0FYC8D/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete0FYC8D/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
img_train.shape,label_train.shape

((50000, 32, 32, 3), (50000,))

In [ ]:
#img_train = img_train[:1000]

In [ ]:
label_train = tf.one_hot(label_train,depth=10)

In [ ]:
# 32 , 32 , 3
def Conv_layer1(x): 
    x = ZeroPadding2D(padding=(1,1))(x) # 34 34 3
    x = Conv2D(filters=16,kernel_size=(3,3),strides=(1,1))(x) # 32 32 16
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    shortcut = x

    x = Conv2D(filters=16,kernel_size=(3,3),padding='same')(x) # 32 32 16
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=16,kernel_size=(3,3),padding='same')(x) # 32 32 16
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=16,kernel_size=(3,3),padding='same' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=16,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=16,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=16,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    return x

def Conv_layer2(x):# 16 16 32
    x = ZeroPadding2D(padding = (1,1))(x) # 34 34 32
    shortcut = x # 32 32 16
    x = Conv2D(filters=32,kernel_size=(3,3),padding='valid',strides=2)(x)  # 31 -3 /2 +1 16 16 32
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x1 = Conv2D(filters=32,kernel_size=(3,3),padding='valid',strides=2)(shortcut)
    x1 = BatchNormalization()(x1)
    x = Add()([x,x1])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x) 
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    return x
# 16 16 32
def Conv_layer3(x):
    x = ZeroPadding2D(padding = (1,1))(x)
    shortcut = x
    x = Conv2D(filters=64,kernel_size=(3,3),padding='valid',strides=2)(x)# 8 8 64
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=64,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x1 = Conv2D(filters=64,kernel_size=(3,3),padding='valid',strides=2)(shortcut)
    x1 = BatchNormalization()(x1)
    x = Add()([x,x1])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=64,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=64,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    shortcut = x
    x = Conv2D(filters=64,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=64,kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x,shortcut])
    x = Activation('relu')(x)
    # fully
    return x

In [ ]:
inputs = Input(shape = (32,32,3),dtype = 'float32')
x = Conv_layer1(inputs)
x = Conv_layer2(x)
x = Conv_layer3(x)
x = GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
resnet20 = tf.keras.models.Model(inputs,outputs)
resnet20.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
#sgd = tf.keras.optimizers.SGD(lr = 0.1)
nadam = tf.keras.optimizers.Nadam(lr=0.01)
resnet20.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])
resnet20.fit(img_train,label_train,batch_size=128, epochs=10,validation_split=0.1)

Epoch 1/10
352/352 [==============================] - 587s 2s/step - loss: 1.8982 - accuracy: 0.3119 - val_loss: 2.0181 - val_accuracy: 0.4044
Epoch 2/10
352/352 [==============================] - 575s 2s/step - loss: 1.0595 - accuracy: 0.6154 - val_loss: 1.2878 - val_accuracy: 0.5782
Epoch 3/10
352/352 [==============================] - 574s 2s/step - loss: 0.7950 - accuracy: 0.7190 - val_loss: 3.3589 - val_accuracy: 0.3672
Epoch 4/10
352/352 [==============================] - 575s 2s/step - loss: 0.6441 - accuracy: 0.7760 - val_loss: 1.4323 - val_accuracy: 0.5810
Epoch 5/10
352/352 [==============================] - 574s 2s/step - loss: 0.5366 - accuracy: 0.8147 - val_loss: 0.9034 - val_accuracy: 0.7008
Epoch 6/10
352/352 [==============================] - 574s 2s/step - loss: 0.4559 - accuracy: 0.8406 - val_loss: 0.8377 - val_accuracy: 0.7338
Epoch 7/10
352/352 [==============================] - 576s 2s/step - loss: 0.3945 - accuracy: 0.8602 - val_loss: 1.2252 - val_accuracy: 0.6574

In [ ]:
resnet20.save("resnet20_nadam.h5")

In [ ]:
label_test = tf.one_hot(label_test,depth=10)

InvalidArgumentError: ignored

In [ ]:
res = resnet20.evaluate(img_test,label_test,verbose=1) # 과대적합 떴다.

313/313 [==============================] - 32s 101ms/step - loss: 0.8066 - accuracy: 0.7497
